In [11]:
%pip install matplotlib
%matplotlib inline
%pip install seaborn
%pip install scipy
%pip install statsmodels
%pip install scikit-learn
%pip install wquantiles
%pip install openpyxl
%pip install opendatasets

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import opendatasets as od
import sklearn
import wquantiles
import os

from pathlib import Path
from scipy.stats import trim_mean
from statsmodels import robust
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.impute import KNNImputer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [13]:
%reload_kedro

[10/25/24 19:06:23] INFO     Resolved project path as:                                              ]8;id=780997;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=247805;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py#178\178]8;;\
                             C:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\base_                
                             monopolio\mono-01.                                                                    
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=987208;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=337598;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/25/24 19:06:24] INFO     Kedro project mono_01                                                  ]8;id=201201;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=742497;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=493397;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=869146;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=658854;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=917803;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\ipython\__init__.py#151\151]8;;\

In [14]:
catalog


{'companies': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/thecr/OneDrive/Escritorio/depocito_mono/monopolio_super/base_monopolio/mono-01/data/01_raw/companies.csv'), "
              "protocol='file', load_args={}, save_args={'index': False})",
 'reviews': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/thecr/OneDrive/Escritorio/depocito_mono/monopolio_super/base_monopolio/mono-01/data/01_raw/reviews.csv'), "
            "protocol='file', load_args={}, save_args={'index': False})",
 'shuttles': "kedro_datasets.pandas.excel_dataset.ExcelDataset(filepath=PurePosixPath('C:/Users/thecr/OneDrive/Escritorio/depocito_mono/monopolio_super/base_monopolio/mono-01/data/01_raw/shuttles.xlsx'), "
             "protocol='file', load_args={'engine': 'openpyxl'}, "
             "save_args={'index': False}, writer_args={'engine': 'openpyxl'})",
 'preprocessed_companies': "kedro_datasets.pandas.parquet_dataset.ParquetDataset(filepath=Pure

In [15]:
df = catalog.load("pos_proceso")
df.head()

[10/25/24 19:06:30] INFO     Loading data from pos_proceso (ParquetDataset)...                  ]8;id=618752;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=413243;file://c:\Users\thecr\OneDrive\Escritorio\depocito_mono\monopolio_super\Lib\site-packages\kedro\io\data_catalog.py#389\389]8;;\

,Id,Subsegmento,Sexo,Region,Edad,Renta,Antiguedad,Internauta,Adicional,Dualidad,Monoproducto,Ctacte,Consumo,Hipotecario,Debito,CambioPin,Cuentas,TC
1,2,160,1,13,46,143640,69,1,0,0,0,1,0,1,0,1,1,1
2,3,170,1,13,45,929106,24,1,1,0,0,1,0,1,1,1,1,2
3,4,151,1,13,46,172447,134,0,1,0,1,0,0,0,0,1,1,2
4,5,170,1,13,46,805250,116,0,1,1,0,1,0,1,0,1,2,3
5,6,170,1,13,47,707664,67,1,1,0,0,1,0,0,1,1,1,2


In [18]:
def split_data(df: pd.DataFrame, parameters: dict) -> tuple:
    X = df[parameters["featrures01"]]
    y = df["Renta"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=parameters["test_nume"])
    return X_train, X_test, y_train, y_test

In [19]:
# Dividir los datos
X_train, X_test, y_train, y_test = split_data(df)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Dividir los datos                                                                          │
│ ❱ 2 X_train, X_test, y_train, y_test = split_data(df)                                            │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: split_data() missing 1 required positional argument: 'parameters'

In [22]:
model = LinearRegression()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

LinearRegression()

In [ ]:
# Crear y ajustar el modelo de Máquina de Soporte Vectorial (SVR)
svr_model = SVR(kernel='rbf')  # Puedes probar otros kernels como 'linear', 'poly', etc.
svr_model.fit(X_train, y_train)
y_pred02 = svr_model.predict(X_test)
mse_02 = mean_squared_error(y_test, y_pred02)
r2_02 = r2_score(y_test, y_pred02)

In [ ]:
# Crear y ajustar el modelo de Árbol de Decisiones
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
# Predecir con los datos de prueba
y_pred_03 = tree_model.predict(X_test)
# Calcular y mostrar las métricas de rendimiento
mse_03 = mean_squared_error(y_test, y_pred_03)
r2_03 = r2_score(y_test, y_pred_03)